In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.applications import MobileNetV2
from sklearn.model_selection import train_test_split
import os

In [ ]:
# 1. Load CIFAR-10 dataset
# CIFAR-10 is already available in TensorFlow datasets, but we can carve out validation data manually from the training data
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()


170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [ ]:
# Normalize pixel values to be between 0 and 1
x_train, x_test = x_train / 255.0, x_test / 255.0

In [ ]:
# Split part of the training data into validation
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.1, random_state=42)


In [ ]:
# 2. Build the MobileNetV2 model
base_model = MobileNetV2(input_shape=(32, 32, 3), include_top=False, weights='imagenet')


<ipython-input-6-7e28e87d7f65>:2: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(input_shape=(32, 32, 3), include_top=False, weights='imagenet')


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:

base_model.trainable = True

In [ ]:
# Build the custom model on top of MobileNetV2
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(512, activation='relu'),
    layers.Dense(10, activation='softmax')
])

In [ ]:
# 3. Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# 4. Create ModelCheckpoint callback to save the best model
# Save the model to Google Drive (make sure you've mounted Google Drive)
google_drive_path = '/content/drive/MyDrive/cifar10_mobilenetv2_best_model.keras'


In [ ]:
checkpoint_callback = ModelCheckpoint(google_drive_path,
                                      save_best_only=True,
                                      monitor='val_loss',
                                      mode='min',
                                      verbose=1)

In [ ]:
# 5. Train the model
history = model.fit(x_train, y_train, epochs=50, validation_data=(x_val, y_val), callbacks=[checkpoint_callback])


Epoch 1/50
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.4272 - loss: 1.7248
Epoch 1: val_loss improved from inf to 2.92089, saving model to /content/drive/MyDrive/cifar10_mobilenetv2_best_model.keras
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 86s 30ms/step - accuracy: 0.4273 - loss: 1.7246 - val_accuracy: 0.3186 - val_loss: 2.9209
Epoch 2/50
1405/1407 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.6643 - loss: 1.0043
Epoch 2: val_loss improved from 2.92089 to 1.25368, saving model to /content/drive/MyDrive/cifar10_mobilenetv2_best_model.keras
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 19s 14ms/step - accuracy: 0.6643 - loss: 1.0042 - val_accuracy: 0.6030 - val_loss: 1.2537
Epoch 3/50
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.7201 - loss: 0.8519
Epoch 3: val_loss did not improve from 1.25368
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 19s 13ms/step - accuracy: 0.7201 - loss: 0.8519 - val_accuracy: 0.6098 - val_loss: 1.3087
Epoch 4/50
1404/1407 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.74

In [ ]:
# 6. Evaluate the model on the test set
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print(f'Test accuracy: {test_acc}')

313/313 - 4s - 14ms/step - accuracy: 0.5615 - loss: 1.4017
Test accuracy: 0.5615000128746033


Loading Model from drive and training for another 150 epoch for the model to converge

In [ ]:
# Load the model
model = tf.keras.models.load_model(google_drive_path)

In [ ]:
# Train for 50 more epochs (epochs 51 to 100)
history = model.fit(
    x_train, y_train,
    epochs=150,
    initial_epoch=50,  # This ensures that training resumes from epoch 51
    validation_data=(x_val, y_val),
    callbacks=[checkpoint_callback]
)


Epoch 51/150
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.8895 - loss: 0.3369
Epoch 51: val_loss improved from inf to 0.57353, saving model to /content/drive/MyDrive/cifar10_mobilenetv2_best_model.keras
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 75s 27ms/step - accuracy: 0.8895 - loss: 0.3369 - val_accuracy: 0.8200 - val_loss: 0.5735
Epoch 52/150
1405/1407 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8965 - loss: 0.3134
Epoch 52: val_loss did not improve from 0.57353
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 17s 12ms/step - accuracy: 0.8964 - loss: 0.3135 - val_accuracy: 0.7722 - val_loss: 0.7004
Epoch 53/150
1406/1407 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.8772 - loss: 0.3709
Epoch 53: val_loss did not improve from 0.57353
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 17s 12ms/step - accuracy: 0.8772 - loss: 0.3709 - val_accuracy: 0.8056 - val_loss: 0.6400
Epoch 54/150
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.9041 - loss: 0.2875
Epoch 54: val_loss did not improve from 0.57353
1407

In [ ]:
# 6. Evaluate the model on the test set
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print(f'Test accuracy: {test_acc}')

313/313 - 5s - 15ms/step - accuracy: 0.7446 - loss: 0.7775
Test accuracy: 0.7445999979972839
